
# Practice SQL with `pandas`, Pt. 1


---

## Review: `pandas` and SQL


### The `pandas` Connector and Functions for SQL

We can leverage SQL through `pandas` using the `pandas.io.sql` module:

```python
import pandas as pd
from pandas.io import sql
```



#### Sql.read_sql(sql, con[, index_col, ...])
- Reads a SQL query or database table into a DataFrame.


#### DataFrame.to_sql(name, con[, flavor, ...])
- Writes records stored in a DataFrame to a SQL database.

### What basic sql operators have we learned?

We're going to be reading data into pandas from CSVs, and writing them out to sqlite databases.  

So first let's just read in the data

In [81]:
import pandas as pd
car_names = pd.read_csv('../datasets/csv/car-names.csv', encoding = 'utf-8')
# Checking what our DataFrame looks like.
car_names.head(3)

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'


### Creating the sqlite connection

We need to specify into which database we want out tables written.  To do this, we set up the connection.

**If no database exists, our command will create one.**

*Keep in mind that the directory your notebook opens is its base directory for all future SQL actions.*

```python
connection = sqlite3.connect('./datasets/sql/Cars.db.sqlite')
```

In [82]:
from pandas.io import sql
import sqlite3
connection = sqlite3.connect('../datasets/sql/Cars.db.sqlite')


#### Convert the loaded `.csv` to a SQL file.  
Because DataFrames are similar to SQL tables, you can now read and convert a `pandas` DataFrame named `car_names` into a SQL table in the newly created SQLite database above.

```python
car_names.to_sql(name = 'car_names', con = connection, if_exists = 'replace', index = False)
```

Important `.to_sql` arguments include:
- `name`: The name of the table; useful if you have multiple tables in a SQL database.
- `con`: The connection path to where the data should be placed.
- `if_exists`: The condition to pass if the table already exists.


In [83]:
# Converts a DataFrame into a SQL database.
car_names.to_sql(name = 'car_names', con = connection, if_exists = 'replace', index = False)

# if you check that directory, you should see the database file

> **Note:** If you wanted a temporary SQL database, using the command below would allow you to access a database store in memory (RAM) as opposed to in storage.

``` python
conn = sqlite3.connect(':memory:')
```

### Let's create a second table in the same database

The table should be called `car_makers`.

In [84]:
car_makers_csv = '../datasets/csv/car-makers.csv'

# Creating a table for order breakdowns.
makers = pd.read_csv(car_makers_csv, encoding = 'utf-8')

makers.to_sql(name = 'car_makers', con = connection, if_exists = 'replace', index = False)

# note that it's the same connection because we're working in the same database

###  Create a third table in the `cars` database for the car data.

The table should be called `car_data`.  Try this yourselves.

In [85]:
car_data_csv = '../datasets/csv/cars-data.csv'

# Creating a table for the sales targets.
data = pd.read_csv(car_data_csv, encoding = 'utf-8')

data.to_sql(name = 'car_data', con = connection, if_exists = 'replace', index = False)


### Let's now read the entire `car_names` table from your SQL database into a DataFrame.

Reading into a DataFrame with a query string can be accomplished using:
```python
# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object.
cars = sql.read_sql(query_string, con = connection)
```

In [86]:
# This is our SQL query:
query = 'select * from car_names'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
results = sql.read_sql(query, con = connection)

results.head()

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'
3,4,'amc','amc rebel sst'
4,5,'ford','ford torino'


## Side Note: Normalized vs. Denormalized Databases

---

There are several ways to organize data in a relational database. Two common definitions for data set ups are **normalized** and **denormalized**.

- __Normalized__ structures include a single table per entity and use many foreign keys or link tables to connect entities together.

- __Denormalized__ tables have fewer tables and may (for example) place all of the tweets and user information in one table.




Each style has its own advantages and disadvantages. 

- Denormalized tables duplicate a lot of information. For example, in a combined tweets/users table, we may store the address of each user. Now, instead of storing this once per user, we are storing it once per tweet!
- However, this makes the data easy to access if we ever need to find the tweet _and_ the user's location.

- Normalized tables save the storage space by separating the information. 
- However, if we ever need to access two pieces of information at once — like in our example — we would need to join the corresponding tables, which can take more time.

### Let's write a Python function to query a database using `pandas` and return a DataFrame.

The function should take two arguments:
- The query string
- The database connection object

In [87]:

CARS = sqlite3.connect('../datasets/sql/Cars.db.sqlite')

def Q(query, db=CARS):
    return sql.read_sql(query, db)

>Check: what would happen if I omitted the first argument?  What about the second?

Select the first five rows of the `car_names` table.

> **Hint**: The LIMIT command in SQL can limit the number of rows returned.

In [88]:
Q('select * from car_names limit 5')

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'
3,4,'amc','amc rebel sst'
4,5,'ford','ford torino'


### Let's add some more cars into the `car_names` table.

The `.execute()` function will come in handy here, executing a SQL command string.
```python
connection.execute()
```

In [89]:
ferrari = (None, 'Ferrari','The Ferrari')
tesla = [None, 'Tesla', None]

In [90]:
CARS.execute('INSERT INTO car_names VALUES (?, ?, ?)', ferrari)
CARS.execute('INSERT INTO car_names VALUES (?, ?, ?)',tesla)

In [91]:
Q('SELECT * FROM car_names').tail()

,Id,Model,Make
403,404.0,'dodge','dodge rampage'
404,405.0,'ford','ford ranger'
405,406.0,'chevrolet','chevy s-10'
406,NaN,Ferrari,The Ferrari
407,NaN,Tesla,None


### Query the `car_names` table for all columns where `'Model' = 'Tesla.'`

In [92]:
Q('SELECT * FROM car_names WHERE "Model" = "Tesla"')

,Id,Model,Make
0,None,Tesla,None


## Side note: capitalization....

Why is "Model" capitalized?  Would it work without it?

In [93]:
Q('SELECT * FROM car_names WHERE model = "Tesla"')

,Id,Model,Make
0,None,Tesla,None


### Select the first five rows of the `car_makers` table.

In [94]:
Q('select * from car_makers limit 5')

,Id,Maker,FullName,Country
0,1,'amc','American Motor Company',1
1,2,'volkswagen','Volkswagen',2
2,3,'bmw','BMW',2
3,4,'gm','General Motors',1
4,5,'ford','Ford Motor Company',1


### Select the first five rows of the `car_data` table.

In [95]:
Q('select * from car_data limit 5')

,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year
0,1,18,8,307.0,130,3504,12.0,1970
1,2,15,8,350.0,165,3693,11.5,1970
2,3,18,8,318.0,150,3436,11.0,1970
3,4,16,8,304.0,150,3433,12.0,1970
4,5,17,8,302.0,140,3449,10.5,1970


## SQL JOIN Types

---

SQL JOINs are used when data is spread out in different tables. A JOIN operation allows us to combine rows from two or more tables in a single new table. In order for this to be possible, a common field need to exists between the tables.

JOIN operations can be considered operations between two sets in which records with the same key are combined and records missing in one set are either discarded or included as NULL values.

YOU HAVE SEEN THIS BEFORE!!  WHERE?



![join types](../images/joins.gif)

JOIN types include:
- **INNER JOIN:** Returns all rows where there is at least one match in BOTH tables.
- **LEFT JOIN:** Returns all rows from the left table and matched rows from the right table.
- **RIGHT JOIN:** Returns all rows from the right table and matched rows from the left table.
- **FULL JOIN:** Returns all rows where there is a match in ONE of the tables.

![sql join types](../images/sql-joins.jpeg)

### In our example, we can use Order ID as the matching feature to perform merges.

Let's check out all of the ways we can merge these tables.

### Practice INNER JOINs

The most common type of JOIN is `SQL INNER JOIN` (SIMPLE JOIN). A `SQL INNER JOIN` returns all rows from multiple tables in which the JOIN condition is met. 

If we `INNER JOIN` on `Id`, it takes the intersection of the two tables, excluding the rows for which `CustomerID` is NULL in EITHER of the two tables.

Essentially, only matching pairs of Order IDs from both data sets will be returned.


In [96]:
# Why doesn't this work?
inner_join = Q('SELECT "Make", "MPG", "Horsepower", "Year" '
'FROM car_names '
'INNER JOIN car_data '
'ON "Id"')
inner_join.head()

DatabaseError: Execution failed on sql 'SELECT "Make", "MPG", "Horsepower", "Year" FROM car_names INNER JOIN car_data ON "Id"': ambiguous column name: Id

In [97]:
inner_join = Q('SELECT car_names."Make", car_data."MPG", car_data."Horsepower", car_data."Year" '
'FROM car_names '
'INNER JOIN car_data '
'ON car_names."Id" = car_data."Id"')
inner_join.head()

,Make,MPG,Horsepower,Year
0,'chevrolet chevelle malibu',18,130,1970
1,'buick skylark 320',15,165,1970
2,'plymouth satellite',18,150,1970
3,'amc rebel sst',16,150,1970
4,'ford torino',17,140,1970


In [98]:
# will this work?  under what circumstances?
# inner_join = Q('SELECT "Make", "MPG", "Horsepower", "Year" '
# 'FROM car_names '
# 'INNER JOIN car_data '
# 'ON car_names."Id" = car_data."Id"')
# inner_join.head()

### Practice LEFT JOINs

The `LEFT JOIN` keyword returns all rows from the left table (`table1`), along with matching rows in the right table (`table2`). When there is no match, the result is NULL on the right side.

**Select `Make`, `MPG`, `Horsepower`, and `Year`**.
- `SELECT FROM` the `car_names` table.
- `LEFT JOIN` the `car_data` table by `Id`.

In [99]:
left_join = Q('SELECT car_names."Make", car_data."MPG", car_data."Horsepower", car_data."Year" '
'FROM car_names '
'LEFT JOIN car_data '
'ON car_names."Id"=car_data."Id"')
left_join

,Make,MPG,Horsepower,Year
0,'chevrolet chevelle malibu',18,130,1970
1,'buick skylark 320',15,165,1970
2,'plymouth satellite',18,150,1970
3,'amc rebel sst',16,150,1970
4,'ford torino',17,140,1970
5,'ford galaxie 500',15,198,1970
6,'chevrolet impala',14,220,1970
7,'plymouth fury iii',14,215,1970
8,'pontiac catalina',14,225,1970
9,'amc ambassador dpl',15,190,1970


###  RIGHT JOINs and FULL OUTER JOINs 

> **Note:** We haven't included exercises for RIGHT and FULL OUTER JOINs, because they are not supported in this example.

The `RIGHT JOIN` keyword would join all rows from the right table (`table2`) with the matching rows in the left table (`table1`). The result is NULL on the left side when there is no match.

>Check: on the whole, people use `RIGHT JOIN` almost never.  Why?

The `FULL OUTER JOIN` keyword returns all rows from the left table (`table1`) and the right table (`table2`). This JOIN combines results from both `LEFT` and `RIGHT` JOINs and all information from both tables into one. You can imagine that this can involve lots of repetitious information and/or NULL values.

## Addtional Resources

---

These resources are a bit long-winded but are good for explaining `pandas` functions from a SQL programmer's perspective:

- [Pydata Video](https://www.youtube.com/watch?v=1uVWjdAbgBg)  
- [Associated GitHub Repo](https://github.com/gjreda/pydata2014nyc/tree/master/data)
- [`pandas` Merge, JOIN, and Concatenate](http://pandas.pydata.org/pandas-docs/stable/merging.html)

## Independent Practice:

- select the MPG, Horspower and Weight columns for cars built in 1970
- return the average horsepower for cars with 8 cylinders
- return a list of Makes for cars produced after 1980


### For these, also do in pandas and check against your SQL queries

In [100]:
# select the MPG, Horspower and Weight columns for cars built in 1970
Q('select MPG, Horsepower, Cylinders from car_data where Year = 1970')

,MPG,Horsepower,Cylinders
0,18,130,8
1,15,165,8
2,18,150,8
3,16,150,8
4,17,140,8
5,15,198,8
6,14,220,8
7,14,215,8
8,14,225,8
9,15,190,8


In [114]:
data[data['Year']==1970][['MPG', 'Horsepower', 'Cylinders']]

,MPG,Horsepower,Cylinders
0,18,130,8
1,15,165,8
2,18,150,8
3,16,150,8
4,17,140,8
5,15,198,8
6,14,220,8
7,14,215,8
8,14,225,8
9,15,190,8


In [35]:
# return the average horsepower for cars with 8 cylinders

Q('select AVG(Horsepower) from car_data where Cylinders = 8')

,AVG(Horsepower)
0,158.453704


In [127]:

data[data['Cylinders']==8]['Horsepower'].astype(float).mean()

158.4537037037037

In [201]:

Q('select Make from car_data left join car_names on car_data.Id = car_names.Id Where Year > 1980')

,Make
0,'plymouth reliant'
1,'buick skylark'
2,'dodge aries wagon (sw)'
3,'chevrolet citation'
4,'plymouth reliant'
5,'toyota starlet'
6,'plymouth champ'
7,'honda civic 1300'
8,'subaru'
9,'datsun 210 mpg'


In [202]:
pd.merge(data, car_names, how='left')[pd.merge(data, car_names, how='left')['Year']>1980]["Make"]

345                     'plymouth reliant'
346                        'buick skylark'
347               'dodge aries wagon (sw)'
348                   'chevrolet citation'
349                     'plymouth reliant'
350                       'toyota starlet'
351                       'plymouth champ'
352                     'honda civic 1300'
353                               'subaru'
354                       'datsun 210 mpg'
355                        'toyota tercel'
356                          'mazda glc 4'
357                   'plymouth horizon 4'
358                       'ford escort 4w'
359                       'ford escort 2h'
360                     'volkswagen jetta'
361                          'renault 18i'
362                        'honda prelude'
363                       'toyota corolla'
364                         'datsun 200sx'
365                            'mazda 626'
366            'peugeot 505s turbo diesel'
367                            'saab 900s'
368        